# This notebook contains code to generate baselines for the import prediction problem that the newural network results could later be compared to

In [1]:
import joblib
from tqdm.notebook import tqdm
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import networkx as nx
import numpy as np

In [2]:
# Different number of negatives a classifier is given. For Target size of 1, the problem becomes a binary
# classifiecation problem (one positive, one negative). In other cases, this is a multiclass classification
TARGET_SIZES = (1, 4, 24, 124)
EMBED_SIZE = 10 # Embedding size used
# keys for different features described below
FEATURES = ("ed_", "emb_self_", "emb_imp_", "degree_", "min_dist_")

In [4]:
test = joblib.load("../data/graphsTest")
train = joblib.load("../data/graphsTrain")

## Calculating various features to be used in baseline prediction. 

### (i) Edit distance between compilation unit names

In [5]:
# Copied from Wikipedia
def levenshtein(s1, s2):
    """
    Function to calculate edit distance between to strings. Copied from wikipedia
    """
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [6]:
def calculate_edit_distance(graph):
    """
    For each datapoint in the dataset calculate the edit distance between relevant pairs of nodes
    :param graph:   a dictionary of the following form: 
                    "annotations" - initial node embeddings. These are composed of two parts. The first one
                    encodes the class name, the second part record external imports in that class\compilation unit
                    "edges"   - a list of tuples of three elements each (source, edge_type, destination)
                    "strings" - a dictionary that maps an id of a node to the fully qualified name of the class
                    "targets_[n]" - list of node indices. The first index is the node to which an import is made,
                    the second index is the node from which the import is mad (the node the baseline has to predict)
                    the following n indices are alternative nodes that the system is presented with and has to
                    distinguish from the positive.
    """
    for n in TARGET_SIZES:
        graph["ed_" + str(n)] = np.zeros(len(graph["targets_" + str(n)]))
        if len(graph["targets_" + str(n)]) == 0:
            continue
        anchor_str = graph["strings"][graph["targets_" + str(n)][0]]
        anchor_str = anchor_str.split('.')[-1]
        for i in range(len(graph["targets_" + str(n)])):
            node_id = graph["targets_" + str(n)][i]
            node_str = graph["strings"][node_id]
            node_str = node_str.split('.')[-1]
            graph["ed_" + str(n)][i] = levenshtein(anchor_str, node_str)

### (ii) distance between corresponding embeddings

In [7]:
def calculate_embedding_distance(graph):
    """
    For each datapoint in the dataset calculate the distance between embeddings of teh two relevant nodes.
    Since these embeddings are composed of two parts that encode conceptually different things (see above), 
    two distances are calculated
    :param graph: see docstring for calculate_edit_distance()
    """
    for n in TARGET_SIZES:
        graph["emb_self_" + str(n)] = np.zeros(len(graph["targets_" + str(n)]))
        graph["emb_imp_" + str(n)] = np.zeros(len(graph["targets_" + str(n)]))
        if len(graph["targets_" + str(n)]) == 0:
            continue
        anchor_emb = graph["annotations"][graph["targets_" + str(n)][0]]
        for i in range(len(graph["targets_" + str(n)])):
            node_id = graph["targets_" + str(n)][i]
            node_emb = graph["annotations"][node_id]
            graph["emb_self_" + str(n)][i] = np.sum((anchor_emb[:EMBED_SIZE] - node_emb[:EMBED_SIZE])**2)  
            graph["emb_imp_" + str(n)][i] = np.sum((anchor_emb[EMBED_SIZE:] - node_emb[EMBED_SIZE:])**2)

### (iii) shortest distance from one node to the other on the graph and (iv) degree of a node

In [8]:
def nx_graph(graph):
    """
    Take a graph as described in the docstring for calculate_edit_distance() and return a networkx representation
    of that graph
    """
    G = nx.Graph()
    for i in range(len(graph["strings"])):
        node = i
        G.add_node(node)
    for edge in graph["edges"]:
        node_from, _, node_to = edge
        G.add_edge(node_from, node_to)
    return G

In [9]:
def calculate_graph_features(graph):
    """
    For each datapoint in the dataset calculate the graph-level features for each pair of relevant nodes.
    These features are (i) shortest distance from one node to the other on the graph and (ii) degree of a node
    :param graph: see docstring for calculate_edit_distance()
    """
    G = nx_graph(graph)
    for n in TARGET_SIZES:
        graph["min_dist_" + str(n)] = np.zeros(len(graph["targets_" + str(n)]))
        graph["degree_" + str(n)] = np.zeros(len(graph["targets_" + str(n)]))
        if len(graph["targets_" + str(n)]) == 0:
            continue
        anchor = graph["targets_" + str(n)][0]
        target = graph["targets_" + str(n)][1]
        G.remove_edge(anchor, target)
        path_lengths = nx.single_source_shortest_path_length(G, anchor)
        G.add_edge(anchor, target)
        not_reachable = max(path_lengths.values()) + 1
        # print(path_lengths, not_reachable)
        for i in range(len(graph["targets_" + str(n)])):
            node = graph["targets_" + str(n)][i]
            graph["min_dist_" + str(n)][i] = path_lengths.get(node, not_reachable)
            graph["degree_" + str(n)][i] = G.degree(node)
            # print(graph["strings"][anchor].split('.')[-1], 
            # graph["strings"][node].split('.')[-1], graph["features_min_dist_" + str(n)][i])

## Calculate all the features for the training and testing sets

In [10]:
for graph in tqdm(test):
    calculate_edit_distance(graph)
    calculate_embedding_distance(graph)
    calculate_graph_features(graph)

In [11]:
for graph in tqdm(train):
    calculate_edit_distance(graph)
    calculate_embedding_distance(graph)
    calculate_graph_features(graph)

## See how good each of the features is for import prediction alone

In [12]:
def take_max_or_min(graph, n, key, ismax=True):
    """
    Given a feature name, a graph, and the number of options to consider (TARGET_SIZE)
    select the option that maximizes or minimizes the feature value and
    return 1 if the option is the correct prediction and 0 otherwise
    :param graph: see docstring for calculate_edit_distance()
    :param n:     the number of options to consider. Should be the element of TARGET_SIZES
    :param key:   the feature to make the prediction by. Should be a string and element of FEATURES list
    ;param ismax: whether to take max or min
    """
    if ismax:
        extreme = np.max(graph[key + str(n)][1:])
    else:
        extreme = np.min(graph[key + str(n)][1:])
    if graph[key + str(n)][1] != extreme:
        return 0
    options = np.where(graph[key + str(n)][1:] == extreme)[0]
    if len(options) > 1:
        return 0
    return 1

In [13]:
def test_baseline(baseline, n):
    """
    Test hoe well a given baseline fares with a given number of options in terms of accuraccy.
    Note that accuraccy is obviously dependent on the number of options. The random baseline in always 1/n
    :param baseline:a function that takes a graph and n as options
    :param n:       number of options to consider
    """
    total = 0
    correct = 0
    for graph in test:
        if len(graph["targets_" + str(n)]) == 0:
            continue
        total += 1
        correct += baseline(graph, n)
    return correct/total

### Below is the breakdown of how well each feature is for predicting imports on its own

In [14]:
for n in TARGET_SIZES:
    print("Testing with %d options:" % n)
    for feature in FEATURES:
        min_result = test_baseline(lambda x, y: take_max_or_min(x, y, feature, ismax=False), n)
        max_result = test_baseline(lambda x, y: take_max_or_min(x, y, feature, ismax=True), n)
        print("\t" + feature + ":\t" + str(round(max(min_result, max_result) * 100, 1)))

Testing with 1 options:
	ed_:	55.1
	emb_self_:	51.1
	emb_imp_:	47.6
	degree_:	72.2
	min_dist_:	33.5
Testing with 4 options:
	ed_:	27.6
	emb_self_:	26.8
	emb_imp_:	24.7
	degree_:	52.1
	min_dist_:	11.9
Testing with 24 options:
	ed_:	10.3
	emb_self_:	7.7
	emb_imp_:	6.6
	degree_:	27.4
	min_dist_:	2.5
Testing with 124 options:
	ed_:	7.3
	emb_self_:	4.0
	emb_imp_:	2.6
	degree_:	16.1
	min_dist_:	2.6


## Normalizing all feature values before feeding them to different classifiers

In [15]:
def normalize(dataset):
    for n in tqdm(TARGET_SIZES):
        for graph in dataset:
            if len(graph["targets_" + str(n)]) == 0:
                continue
            for feature in FEATURES:
                max_value = np.max(graph[feature + str(n)][1:])
                if max_value != 0:
                    graph[feature + str(n)] /= max_value
                    graph[feature + str(n)] -= 0.5

In [16]:
normalize(train)

In [17]:
normalize(test)

## Convert all the data in the format that can be easily used with sklearn classifiers

In [18]:
train_y = {}
train_X = {}
for n in tqdm(TARGET_SIZES):
    train_y[n] = []
    train_X[n] = []
    for graph in train:
        if len(graph["targets_" + str(n)]) == 0:
            continue
        features = np.stack([graph[feature + str(n)] for feature in FEATURES], axis=1)
        for i in range(1, n+2):
            train_X[n].append(features[i])  
        curr_y = np.zeros(n+1)
        curr_y[0] = 1
        train_y[n] += list(curr_y)

## Train random forest and SVM classifiers

In [19]:
classsifiers = {}
for n in tqdm(TARGET_SIZES):
    classsifiers[n] = RandomForestClassifier(n_estimators = 100, n_jobs=2)
    classsifiers[n].fit(train_X[n], train_y[n])

In [20]:
classifierSVM = svm.SVC(probability=True, verbose=True)
classifierSVM.fit(train_X[1], train_y[1])

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [21]:
def test_classifier(graph, n, classifier):
    """
    Test a sklearn classifier. This is analagoues to take_max_or_min function above. 
    The function return 1 if the classifier makes a correct prediction for the given graph and 0 otherwise
    """
    features = np.stack([graph[feature + str(n)] for feature in FEATURES], axis=1)
    test_X = []
    for i in range(1, n+2):
        test_X.append(features[i]) 
    proba = classifier.predict_proba(test_X)
    extreme = np.max(proba, axis=0)[1]
    if proba[0][1] != extreme:
        return 0
    options = np.where(proba[:, 1] == extreme)[0]
    if len(options) > 1:
        return 0
    return 1

## Test random forest and SVM classifiers

In [25]:
for n in TARGET_SIZES:
    print("Testing with %d options:" % n)
    print("Testing with random forest yields:" + 
          str(test_baseline(lambda x, y: test_classifier(x, y, classsifiers[y]), n)))

Testing with 1 options:
Testing with random forest yields:0.755883710198431
Testing with 4 options:
Testing with random forest yields:0.5435724602792489
Testing with 24 options:
Testing with random forest yields:0.2568951930654058
Testing with 124 options:
Testing with random forest yields:0.16117216117216118


In [26]:
for n in TARGET_SIZES:
    print("Testing with %d options:" % n)
    print("Testing with svm yields:" + 
          str(test_baseline(lambda x, y: test_classifier(x, y, classifierSVM), n)))

Testing with 1 options:
Testing with svm yields:0.7641901245962159
Testing with 4 options:
Testing with svm yields:0.45931632161771785
Testing with 24 options:
Testing with svm yields:0.136327817178881
Testing with 124 options:
Testing with svm yields:0.03296703296703297
